In [1]:
import pandas as pd
import swifter
import numpy as np
import string
import matplotlib.pyplot as plt
import seaborn as sns
import nltk

from spacy.lemmatizer import Lemmatizer
from spacy.lang.it.lemmatizer import LOOKUP
from spacy.lang.it.stop_words import STOP_WORDS

from scipy.sparse import save_npz, load_npz

from sklearn.cluster import KMeans, SpectralClustering
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances

import spacy

from scipy.spatial.distance import cdist

import pickle

from gensim import corpora, models
%matplotlib inline

In [2]:
agricole = pd.read_excel('agricole.xlsx', 
                         dtype={'Partita IVA': str, 'Codice fiscale': str}, 
                         sheet_name='Risultati', na_values=['n.d.', 'n.s.'])

multi = pd.read_excel('multi.xlsx', 
                         dtype={'Partita IVA': str, 'Codice fiscale': str}, 
                         sheet_name='Risultati', na_values=['n.d.', 'n.s.'])

In [3]:
multi = multi.append(agricole, ignore_index=True)

In [4]:
multi.columns = multi.columns.str.replace('\n', ' ')

In [5]:
rename = {
'Altri ricavi EUR Ultimo anno disp.': 'other_sales',
'PFN/EBITDA Ultimo anno disp.': 'leva',
'PFN EUR Ultimo anno disp.': 'pfn',
'Durata media dei crediti al lordo IVA (gg) Ultimo anno disp.': 'crediti',
'Durata media dei debiti al lordo IVA (gg) Ultimo anno disp.': 'debiti',
'EBITDA EUR Ultimo anno disp.': 'ebitda',
'Capitale sociale EUR Ultimo anno disp.': 'capitale',
'Giac. media delle scorte (gg) Ultimo anno disp.': 'magazzino',
'MORE Probabilità di default - attuale  %': 'more',
'CRIF score - attuale': 'crif',
'Descrizione di consolidamento': 'cons',
'Ragione sociale' : 'nome', 
'Partita IVA': 'piva',
'Codice fiscale': 'cf',
'Ricavi vendite e prestazioni EUR Ultimo anno disp.': 'sales',
'Oggetto sociale': 'oggetto', 
'ATECO 2007 codice': 'ateco', 
}

In [6]:
multi = multi.rename(columns=rename)

In [7]:
multi = multi[[k for k in rename.values()]]

In [8]:
multi.ateco = multi.ateco.astype(str).str.zfill(6)

In [9]:
def split_ateco(str):
    return [str[start:start + 2] for start in range(0, len(str), 2)]

In [10]:
ateco = multi.ateco.swifter.apply(split_ateco)

C:\Users\AlanMarazzi\Anaconda3\envs\nlp\lib\site-packages\tqdm\std.py:648: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [11]:
multi['ateco_1'] = [i[0] for i in ateco]
multi['ateco_2'] = [i[1][0] for i in ateco]
multi['ateco_3'] = [i[1][1] for i in ateco]
multi['ateco_4'] = [i[2] for i in ateco]

In [12]:
multi.ateco_1.value_counts()

10    5639
01    3270
03     160
Name: ateco_1, dtype: int64

In [13]:
multi = multi[~multi.oggetto.isnull()]

In [14]:
stable = str.maketrans({key: ' ' for key in string.punctuation})

In [15]:
def clean_str_simple(s):
    tokens = nltk.word_tokenize(s.translate(stable), language='italian')
    tokens = [i for i in tokens if i.lower() not in set(nltk.corpus.stopwords.words('italian') + ['d'])]
    return tokens

In [16]:
def print_topics(model, vectorizer, n_top_words):
    words = vectorizer.get_feature_names()
    for topic_idx, topic in enumerate(model.components_):
        print("\nTopic #%d:" % topic_idx)
        print(" ".join([words[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))

In [17]:
nlp = spacy.load("it_core_news_sm")

In [18]:
custom = ['n.', "nell'", "1)", "2)", "3)", "4)", "5)",
          "6)", "7)", "8)", "9)", "a)", "b)", "c)",
          "d)", "e)", "f)", "g)", "h)", "i)", "l)",
          "1.", "2.", "3.", "4.", "5.", "6.", "7."]

common = ['attività', 'società', 'oggetto', 'esercizio', 'scopi', 'articolo', 'legge',
          'organizzazione', 'obiettivi', 'sensi', 'genere']

common_l = []

for i in common:
    for j in ['à', 'è', 'ù', 'ò', 'ì']:
        if j in i:
            common_l.append(i.replace(j, ''))
    common_l.append(i)

for c in custom:
    STOP_WORDS.add(c)
    
for c in common_l:
    STOP_WORDS.add(c)
    
for w in STOP_WORDS:
    nlp.vocab[w].is_stop = True

In [19]:
accenti = {
    "a": "à",
    "e": "è",
    "u": "ù",
    "o": "ò",
    "i": "ì"
}

In [27]:
def apostrofo_to_accento(s):
    t = s.lower().replace('- ', '').split()
    for i, w in enumerate(t):
        if "'" == w[-1] and len(w) > 2 and w[-2] in 'aeiou':
            n = w[:-2]
            t[i]  = n + accenti[w[-2]]
    return ' '.join(t)

def clean_simple(s):
    clean = apostrofo_to_accento(s)
    p = nlp(clean)
    return [w.text for w in p if len(w) >= 3 
            and w.pos_ == 'NOUN' 
            and not w.is_stop 
            and not w.is_punct
            and not w.text.isdigit()]

def count(ser, strategy):
    p = ser.swifter.apply(strategy)
    count = CountVectorizer()
    return count.fit_transform([' '.join(s) for s in p]), count.get_feature_names(), count

def tfidf_count(ser, strategy):
    p = ser.swifter.apply(strategy)
    count = TfidfVectorizer(strip_accents='unicode', max_df=.5, min_df=.05, ngram_range=(2, 2))
    return count.fit_transform([' '.join(s) for s in p]), count.get_feature_names(), count

In [28]:
%%time
c, c_names, counter = count(multi.oggetto, clean_simple)

C:\Users\AlanMarazzi\Anaconda3\envs\nlp\lib\site-packages\tqdm\std.py:648: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel



Wall time: 7min


In [29]:
%%time
tfidf, tfidf_names, tfidf_counter = tfidf_count(multi.oggetto, clean_simple)

C:\Users\AlanMarazzi\Anaconda3\envs\nlp\lib\site-packages\tqdm\std.py:648: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel



Wall time: 7min


In [30]:
save_npz('c.npz', c)
save_npz('tfidf.npz', tfidf)

In [31]:
def pickle_it(obj, path):
    with open(path, "wb") as f:
        pickle.dump(obj, f)
    return True

In [32]:
pickle_it([c_names, counter], "c_names.pkl")
pickle_it([tfidf_names, tfidf_counter], "tfidf_names.pkl")

True

In [33]:
def unpickle_it(path):
    with open(path, "rb") as f:
        p = pickle.load(f)
    return p

def read_models(name):
    npz = load_npz(name + '.npz')
    pkl = unpickle_it(name + '_names.pkl')
    return npz, pkl

names = ["c", "tfidf"]

c, [c_names, counter] = read_models(names[0])
tfidf_count, [tfidf_names, tfidf_counter] = read_models(names[1])

In [85]:
%%time
lda_count = LatentDirichletAllocation(30, max_iter=500, n_jobs=6, 
                                       verbose=1, evaluate_every=20,
                                      perp_tol=.1, random_state=12346)
lda_count.fit(c)

iteration: 1 of max_iter: 500
iteration: 2 of max_iter: 500
iteration: 3 of max_iter: 500
iteration: 4 of max_iter: 500
iteration: 5 of max_iter: 500
iteration: 6 of max_iter: 500
iteration: 7 of max_iter: 500
iteration: 8 of max_iter: 500
iteration: 9 of max_iter: 500
iteration: 10 of max_iter: 500
iteration: 11 of max_iter: 500
iteration: 12 of max_iter: 500
iteration: 13 of max_iter: 500
iteration: 14 of max_iter: 500
iteration: 15 of max_iter: 500
iteration: 16 of max_iter: 500
iteration: 17 of max_iter: 500
iteration: 18 of max_iter: 500
iteration: 19 of max_iter: 500
iteration: 20 of max_iter: 500, perplexity: 445.8542
iteration: 21 of max_iter: 500
iteration: 22 of max_iter: 500
iteration: 23 of max_iter: 500
iteration: 24 of max_iter: 500
iteration: 25 of max_iter: 500
iteration: 26 of max_iter: 500
iteration: 27 of max_iter: 500
iteration: 28 of max_iter: 500
iteration: 29 of max_iter: 500
iteration: 30 of max_iter: 500
iteration: 31 of max_iter: 500
iteration: 32 of max_iter:

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=20, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=500,
                          mean_change_tol=0.001, n_components=30, n_jobs=6,
                          perp_tol=0.1, random_state=12346,
                          topic_word_prior=None, total_samples=1000000.0,
                          verbose=1)

In [92]:
%%time
summary = {}

for i in [30, 50, 70, 100]:
    print(i)
    perps = []
    lda = LatentDirichletAllocation(i, max_iter=300, n_jobs=6, 
                                    verbose=0, evaluate_every=30, 
                                    random_state=1234)
    lda.fit(tfidf)
    p = lda.perplexity(tfidf)
    perps.append(p)
    print('Perplexity: ', p)
    print()
    summary[i] = lda

30
30 6412.404923567844

50
50 11068.878800450226

70
70 15578.722968283673

100
100 25654.734355974502

Wall time: 8min 55s


In [34]:
%%time
lda_tfidf = LatentDirichletAllocation(50, max_iter=500, n_jobs=6, 
                                       verbose=1, evaluate_every=20, 
                                       random_state=12346)
lda_tfidf.fit(tfidf_count)

iteration: 1 of max_iter: 500
iteration: 2 of max_iter: 500
iteration: 3 of max_iter: 500
iteration: 4 of max_iter: 500
iteration: 5 of max_iter: 500
iteration: 6 of max_iter: 500
iteration: 7 of max_iter: 500
iteration: 8 of max_iter: 500
iteration: 9 of max_iter: 500
iteration: 10 of max_iter: 500
iteration: 11 of max_iter: 500
iteration: 12 of max_iter: 500
iteration: 13 of max_iter: 500
iteration: 14 of max_iter: 500
iteration: 15 of max_iter: 500
iteration: 16 of max_iter: 500
iteration: 17 of max_iter: 500
iteration: 18 of max_iter: 500
iteration: 19 of max_iter: 500
iteration: 20 of max_iter: 500, perplexity: 437.3053
iteration: 21 of max_iter: 500
iteration: 22 of max_iter: 500
iteration: 23 of max_iter: 500
iteration: 24 of max_iter: 500
iteration: 25 of max_iter: 500
iteration: 26 of max_iter: 500
iteration: 27 of max_iter: 500
iteration: 28 of max_iter: 500
iteration: 29 of max_iter: 500
iteration: 30 of max_iter: 500
iteration: 31 of max_iter: 500
iteration: 32 of max_iter:

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=20, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=500,
                          mean_change_tol=0.001, n_components=50, n_jobs=6,
                          perp_tol=0.1, random_state=12346,
                          topic_word_prior=None, total_samples=1000000.0,
                          verbose=1)

In [35]:
pickle_it(lda_count, 'lda_count.pkl')
pickle_it(lda_tfidf, 'lda_tfidf.pkl')

NameError: name 'lda_count' is not defined

In [ ]:
lda_count = unpickle_it('lda_count.pkl')
lda_tfidf = unpickle_it('lda_tfidf.pkl')

In [25]:
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print ("Topic %d:" % (topic_idx))
        print (" ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

no_top_words = 5
display_topics(lda_count, c_names, no_top_words)

NameError: name 'lda_count' is not defined

In [36]:
display_topics(lda_tfidf, tfidf_names, no_top_words)

Topic 0:
tecniche produzione vendita prodotti fini speculazione valorizzazione prodotti dell attivita
Topic 1:
prodotti soci prodotti sottoprodotti commercializzazione prodotti trasformazione prodotti lavorazione trasformazione
Topic 2:
allevamento animali coltivazione fondo dell attivita fideiussioni garanzie prodotti prodotti
Topic 3:
partecipazioni interessenze interessenze imprese operazioni mobiliari fideiussioni garanzie operazioni finanziarie
Topic 4:
all articolo prodotti soci soci cooperativa requisiti interessi prodotti sottoprodotti
Topic 5:
import export vendita prodotti conto conto interessenze partecipazioni interessenze imprese
Topic 6:
commercio prodotti prodotti prodotti interessenze partecipazioni partecipazioni imprese dettaglio prodotti
Topic 7:
prodotti produzione commercializzazione prodotti vendita prodotti fidejussioni avalli dell attivita
Topic 8:
produzione vendita vendita prodotti partecipazioni imprese interessenze partecipazioni operazioni finanziarie
Topic

In [100]:
for topic_idx, topic in enumerate(summary[100].components_):
    if 'vino' in ' '.join([tfidf_names[i] for i in topic.argsort()[:-20 - 1:-1]]):
        print(topic_idx)

28
68


In [26]:
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

In [27]:
pyLDAvis.sklearn.prepare(lda_count, c, counter)

C:\Users\AlanMarazzi\Anaconda3\envs\nlp\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
36    -0.143251  0.206585       1        1  8.059452
44     0.140120  0.143587       2        1  5.397092
16     0.000699  0.099538       3        1  5.080556
8     -0.062488  0.191617       4        1  4.430000
17    -0.217978  0.179701       5        1  4.224119
10     0.055690  0.123238       6        1  3.875955
37     0.077514  0.127219       7        1  3.782601
24     0.158090 -0.015268       8        1  3.388976
18     0.059675  0.054474       9        1  3.009298
12    -0.258089  0.151042      10        1  2.891947
43     0.219474  0.015568      11        1  2.840363
22    -0.182978  0.103666      12        1  2.705105
42     0.086490  0.004123      13        1  2.653509
32     0.187938  0.024063      14        1  2.640382
33    -0.002569 -0.003052      15        1  2.617170
13     0.215587 -0.017485      16        1  2.444882
7     -0.007774  0.166208      17        1  2.274648
48     0.032580  0.139736      18        1  2.161794
29     0.175315 -0.017858      19        1  1.823490
4     -0.082675  0.070710      20        1  1.784284
9      0.120282  0.077114      21        1  1.683177
5      0.153111 -0.115448      22        1  1.603367
27     0.108830 -0.070589      23        1  1.576445
14     0.030344 -0.084936      24        1  1.561998
47     0.126472  0.101518      25        1  1.548328
41     0.242646  0.014602      26        1  1.548290
2      0.142529 -0.051953      27        1  1.526078
49     0.011300  0.122222      28        1  1.515440
23    -0.008839  0.119681      29        1  1.458687
40     0.181143 -0.036321      30        1  1.440575
11     0.147203 -0.068732      31        1  1.400291
46    -0.036879 -0.184867      32        1  1.331808
30    -0.252630  0.056126      33        1  1.307757
0      0.009786  0.130332      34        1  1.245863
1     -0.028631 -0.170035      35        1  1.244236
3     -0.229179  0.028938      36        1  1.218671
19    -0.150522  0.017107      37        1  1.212744
39    -0.252581  0.045574      38        1  1.201387
34     0.055399 -0.056487      39        1  0.938318
35    -0.016779 -0.213924      40        1  0.853154
6     -0.000205 -0.194336      41        1  0.737836
25     0.098999  0.045918      42        1  0.697305
26    -0.220870 -0.104317      43        1  0.675176
38    -0.099313 -0.006894      44        1  0.554082
20    -0.109167 -0.188225      45        1  0.417726
45    -0.044771 -0.026555      46        1  0.392571
31    -0.114774 -0.200856      47        1  0.317093
21    -0.107079 -0.226872      48        1  0.314346
15    -0.095594 -0.257441      49        1  0.279288
28    -0.111602 -0.247754      50        1  0.112337, topic_info=     Category          Freq         Term         Total  loglift  logprob
6962  Default  21522.000000     prodotti  21522.000000  30.0000  30.0000
8308  Default   8623.000000         soci   8623.000000  29.0000  29.0000
4925  Default   3196.000000        latte   3196.000000  28.0000  28.0000
6982  Default  12112.000000   produzione  12112.000000  27.0000  27.0000
2490  Default   4391.000000  cooperativa   4391.000000  26.0000  26.0000
...       ...           ...          ...           ...      ...      ...
5622  Topic50      3.372758     modalità    551.893241   1.6938  -5.1890
7345  Topic50      1.946137  recepimento     65.006748   3.2828  -5.7389
6200  Topic50      1.930716      ottobre     88.421937   2.9672  -5.7469
3327  Topic50      2.802948         enti   2528.999167  -0.0135  -5.3741
244   Topic50      2.479836     acquisto   2252.028919  -0.0200  -5.4966

[3182 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
8        18  0.846227          102
25       42  0.676835           14
26        4  0.930314          142
37       33  0.908797         1815
40       43  0.988981          197
...     ...       ...          ...
9732   

In [50]:
pyLDAvis.sklearn.prepare(lda_tfidf, tfidf, tfidf_counter)

C:\Users\AlanMarazzi\Anaconda3\envs\nlp\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
36    -0.143251  0.206585       1        1  8.059452
44     0.140120  0.143587       2        1  5.397092
16     0.000699  0.099538       3        1  5.080556
8     -0.062488  0.191617       4        1  4.430000
17    -0.217978  0.179701       5        1  4.224119
10     0.055690  0.123238       6        1  3.875955
37     0.077514  0.127219       7        1  3.782601
24     0.158090 -0.015268       8        1  3.388976
18     0.059675  0.054474       9        1  3.009298
12    -0.258089  0.151042      10        1  2.891947
43     0.219474  0.015568      11        1  2.840363
22    -0.182978  0.103666      12        1  2.705105
42     0.086490  0.004123      13        1  2.653509
32     0.187938  0.024063      14        1  2.640382
33    -0.002569 -0.003052      15        1  2.617170
13     0.215587 -0.017485      16        1  2.444882
7     -0.007774  0.166208      17        1  2.274648
48     0.032580  0.139736      18        1  2.161794
29     0.175315 -0.017858      19        1  1.823490
4     -0.082675  0.070710      20        1  1.784284
9      0.120282  0.077114      21        1  1.683177
5      0.153111 -0.115448      22        1  1.603367
27     0.108830 -0.070589      23        1  1.576445
14     0.030344 -0.084936      24        1  1.561998
47     0.126472  0.101518      25        1  1.548328
41     0.242646  0.014602      26        1  1.548290
2      0.142529 -0.051953      27        1  1.526078
49     0.011300  0.122222      28        1  1.515440
23    -0.008839  0.119681      29        1  1.458687
40     0.181143 -0.036321      30        1  1.440575
11     0.147203 -0.068732      31        1  1.400291
46    -0.036879 -0.184867      32        1  1.331808
30    -0.252630  0.056126      33        1  1.307757
0      0.009786  0.130332      34        1  1.245863
1     -0.028631 -0.170035      35        1  1.244236
3     -0.229179  0.028938      36        1  1.218671
19    -0.150522  0.017107      37        1  1.212744
39    -0.252581  0.045574      38        1  1.201387
34     0.055399 -0.056487      39        1  0.938318
35    -0.016779 -0.213924      40        1  0.853154
6     -0.000205 -0.194336      41        1  0.737836
25     0.098999  0.045918      42        1  0.697305
26    -0.220870 -0.104317      43        1  0.675176
38    -0.099313 -0.006894      44        1  0.554082
20    -0.109167 -0.188225      45        1  0.417726
45    -0.044771 -0.026555      46        1  0.392571
31    -0.114774 -0.200856      47        1  0.317093
21    -0.107079 -0.226872      48        1  0.314346
15    -0.095594 -0.257441      49        1  0.279288
28    -0.111602 -0.247754      50        1  0.112337, topic_info=     Category          Freq         Term         Total  loglift  logprob
6962  Default  21522.000000     prodotti  21522.000000  30.0000  30.0000
8308  Default   8623.000000         soci   8623.000000  29.0000  29.0000
4925  Default   3196.000000        latte   3196.000000  28.0000  28.0000
6982  Default  12112.000000   produzione  12112.000000  27.0000  27.0000
2490  Default   4391.000000  cooperativa   4391.000000  26.0000  26.0000
...       ...           ...          ...           ...      ...      ...
5622  Topic50      3.372758     modalità    551.893241   1.6938  -5.1890
7345  Topic50      1.946137  recepimento     65.006748   3.2828  -5.7389
6200  Topic50      1.930716      ottobre     88.421937   2.9672  -5.7469
3327  Topic50      2.802948         enti   2528.999167  -0.0135  -5.3741
244   Topic50      2.479836     acquisto   2252.028919  -0.0200  -5.4966

[3182 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
8        18  0.846227          102
25       42  0.676835           14
26        4  0.930314          142
37       33  0.908797         1815
40       43  0.988981          197
...     ...       ...          ...
9732   

In [29]:
topics_count = lda_count.transform(c)

In [30]:
multi['lda_count'] = [t.argmax() for t in topics_count]

In [38]:
def sample(obj):
    if obj.shape[0] >= 5:
        return obj.loc[np.random.choice(obj.index, 5, False),:]
    else:
        return obj.loc[np.random.choice(obj.index, obj.shape[0], False),:]

topics_multi = multi.groupby('lda_count', as_index=False).apply(sample)

In [41]:
topics_multi[['lda_count', 'nome', 'piva', 'cf', 'ateco', 'oggetto']].to_excel('with_topics.xlsx', index=False)

In [44]:
multi.lda_count.value_counts()

36    1533
17     599
16     587
44     568
8      388
12     364
10     363
24     352
37     313
22     306
43     240
32     240
13     227
18     198
7      179
29     174
33     173
4      152
5      150
42     131
9      116
48     113
23     110
30     108
41     100
11      97
2       97
47      86
40      85
27      83
14      77
39      75
34      73
0       70
46      69
3       62
49      61
1       55
19      35
38      35
25      31
35      21
45      20
26      12
6       10
31       8
15       4
20       3
21       1
Name: lda_count, dtype: int64

In [46]:
multi[multi.lda_count == 21]['nome']

128    MARICOLTURA DI ROSIGNANO SOLVAY S.R.L.
Name: nome, dtype: object

In [123]:
pars = [
    {'max_df':.5, 'min_df':.05},
    {'max_df':.5, 'min_df':.1},
    {'max_df':.3, 'min_df':.05},
    {'max_df':.3, 'min_df':.01},
    {'max_df':.5, 'min_df':.05, 'sublinear_tf':True},
]

def tfidf_comp(ser, strategy, params_dict):
    p = ser.swifter.apply(strategy)
    count = TfidfVectorizer(**params_dict, strip_accents='unicode')
    return count.fit_transform([' '.join(s) for s in p]), count.get_feature_names(), count

In [125]:
def collect_topics(model, feature_names, no_top_words):
    tps = {}
    for topic_idx, topic in enumerate(model.components_):
        tps[topic_idx] = " ".join([feature_names[i]
                                  for i in topic.argsort()[:-no_top_words - 1:-1]])
    return tps

In [135]:
results = []
for p in pars:
    tfidf, tfidf_names, tfidf_counter = tfidf_comp(multi.oggetto, clean_simple, p)
    lda = LatentDirichletAllocation(50, max_iter=500, n_jobs=6, 
                                    verbose=0, evaluate_every=30, 
                                    random_state=12346)
    lda.fit(tfidf)
    results.append({'counts': [tfidf, tfidf_names, tfidf_counter],
                    'params': p,
                    'model': lda,
                    'topics': collect_topics(lda, tfidf_names, 10)})

C:\Users\AlanMarazzi\Anaconda3\envs\nlp\lib\site-packages\tqdm\std.py:648: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


C:\Users\AlanMarazzi\Anaconda3\envs\nlp\lib\site-packages\tqdm\std.py:648: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


C:\Users\AlanMarazzi\Anaconda3\envs\nlp\lib\site-packages\tqdm\std.py:648: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


C:\Users\AlanMarazzi\Anaconda3\envs\nlp\lib\site-packages\tqdm\std.py:648: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


C:\Users\AlanMarazzi\Anaconda3\envs\nlp\lib\site-packages\tqdm\std.py:648: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [142]:
import pprint

pp = pprint.PrettyPrinter()

for i in results:
    pp.pprint(i['params'])
    pp.pprint({k: ' '.join(v.split()[:5]) for k, v in i['topics'].items()})
    print()

{'max_df': 0.5, 'min_df': 0.05}
{0: 'mercati acque dettaglio attivita uso',
 1: 'conseguimento interessenze mobiliari imprese norme',
 2: 'mutui carattere assunzione confronti garanzie',
 3: 'vendita consorzi fidejussioni fideiussioni export',
 4: 'esclusione rilascio confronti raggiungimento integrazioni',
 5: 'energia impianti coltivazione realizzazione allevamento',
 6: 'campo ricerca alimentazione commercio vendita',
 7: 'settori settore ricerca commercializzazione conto',
 8: 'vendita consorzi fidejussioni fideiussioni export',
 9: 'stabilimenti commercializzazione lavorazione conservazione conto',
 10: 'importazione esportazione commercio materie rappresentanze',
 11: 'investimento collocamento avalli esclusione raggiungimento',
 12: 'salumi stagionatura formaggi carni commercio',
 13: 'risparmio pubblico raccolta imprese garanzie',
 14: 'fondi allevamento coltivazione bestiame terreni',
 15: 'locazione immobili costruzione vendita acquisto',
 16: 'soci cooperativa mutualita cond